In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [2]:
pd.options.display.max_columns=None

In [3]:
df_esios=pd.read_csv('../data/datos_esios.csv', sep=';')

In [4]:
df_esios.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42


In [5]:
df_esios.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2
count,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000
mean,5476.657615,28672.335722,4413.087348,4421.192005,4189.052822,48.745913,42.104412,48.474281,48.483520
std,3185.745755,4615.591310,2217.773186,474.661673,469.408867,14.950758,20.792168,15.091625,15.106957
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.000000,0.000000
25%,3015.600000,24733.475000,2318.000000,4125.800000,3797.900000,40.140000,29.747500,40.050000,40.100000
50%,4820.250000,28921.700000,4636.800000,4483.200000,4148.100000,49.565000,39.170000,49.000000,48.970000
75%,7285.800000,32265.825000,6286.600000,4799.700000,4705.300000,59.450000,51.560000,58.900000,58.742500
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.490000,103.610000


In [6]:
df_esios['datetime'].describe()  #tenemos repetidos los datos para el dia de 25 horas.

count                   26328
unique                  26325
top       2016-10-30 02:00:00
freq                        2
Name: datetime, dtype: object

In [7]:
df_esios['datetime']=pd.to_datetime(df_esios['datetime'])
df_esios.dtypes

datetime              datetime64[ns]
P48_EOLICA                   float64
P48_DEMANDA                  float64
P48_CARBON                   float64
POT_DISP_HULLA_ANT           float64
POT_DISP_HULA_SB             float64
PRECIO_MD_ESP                float64
PRECIO_MD_FR                 float64
PRECIO_I1                    float64
PRECIO_I2                    float64
dtype: object

#### 1) Columna a predecir:

"reservar"

Nuestra clasificación será:

* Si PMI1 > PMD -> 0

* Si PMI1 < PMD -> 1

* Si PMI1 == PMD:

    * Si PMI2 > PMD -> 0
    * Si PMI2 <= PMD -> 1

En la descarga y exploración de datos esios se obtuvo que de los 26.328 datos, hay 3.498 casos en los que el precio del mercado diario coincidió con el del intra, y de esos, 794 horas en las que además el intradiario 2 tampoco subió/bajó con respecto al diario.

In [8]:
df_esios_precio=df_esios[['PRECIO_MD_ESP','PRECIO_I1','PRECIO_I2','PRECIO_MD_FR']]

In [10]:
def reserva(dataframe_precios):
    reserva=np.zeros(len(dataframe_precios['PRECIO_MD_ESP']))
    for k,v in dataframe_precios.iterrows():
        a=v['PRECIO_MD_ESP']
        b=v['PRECIO_I1']
        c=v['PRECIO_I2']
        if (a<b) | ((a==b) & (c>b)):
            reserva[k]=0
        else:
            reserva[k]=1  
    return reserva

In [11]:
df_esios['RESERVA'] = reserva(df_esios_precio)

In [12]:
df_esios.sample(10)  #comprobamos aleatoriamente que ok

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA
24830,2018-07-30 14:00:00,1875.7,35761.9,5267.2,4315.8,4705.3,67.55,53.55,60.56,63.95,1.0
15297,2017-06-28 09:00:00,12024.4,33109.3,3502.0,4794.7,4147.8,51.57,41.99,46.95,51.57,1.0
19115,2017-12-04 10:00:00,3758.9,38352.2,7868.7,4442.0,4705.3,78.20,97.02,80.00,80.85,0.0
3258,2016-02-12 17:00:00,15636.8,32074.3,1505.6,4971.5,4705.3,24.10,34.56,23.10,20.00,1.0
5638,2016-05-21 22:00:00,7248.3,27206.2,709.3,3931.5,3784.3,27.00,27.00,32.00,33.90,0.0
11171,2017-01-07 10:00:00,3280.1,30100.2,6067.8,4127.4,3226.4,73.48,73.48,71.63,69.75,1.0
9612,2016-11-03 11:00:00,1338.3,31350.6,6991.0,3945.9,4143.3,65.89,65.89,65.89,65.89,1.0
13981,2017-05-04 13:00:00,3758.0,31723.7,5488.5,4128.1,3365.1,51.04,47.02,52.06,52.04,0.0
10305,2016-12-02 08:00:00,1780.3,34374.1,7942.0,4403.6,4373.3,71.61,92.97,71.61,72.61,0.0
7909,2016-08-24 13:00:00,4397.5,35708.3,6072.8,3946.2,4705.3,43.05,36.50,43.51,43.51,0.0


#### 2) Interconexión

Creamos columna ACOPLADO_FR:

* Si Precio FR MD == Precio España MD están acoplados (1) si no no (0)*

In [13]:
def interconexion(dataframe_precios_inter):
    acoplado=np.zeros(len(dataframe_precios_inter['PRECIO_MD_ESP']))
    for k,v in dataframe_precios_inter.iterrows():
        a=v['PRECIO_MD_ESP']
        b=v['PRECIO_MD_FR']
        if (a==b):
            acoplado[k]=1
        else:
            acoplado[k]=0  
    return acoplado

In [14]:
df_esios['ACOPLADO_FR']=interconexion(df_esios_precio)

In [15]:
df_esios.sample(10) #comprobamos aleatoriamente que ok

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR
19872,2018-01-04 23:00:00,11595.0,27996.0,992.0,4799.7,4705.3,40.00,31.50,40.00,40.98,0.0,0.0
18411,2017-11-05 02:00:00,9940.3,20240.2,1704.0,4116.2,3433.7,31.00,31.00,35.01,36.50,0.0,1.0
23116,2018-05-20 04:00:00,3104.9,20277.0,2204.1,4263.9,4353.9,59.45,6.11,60.43,59.09,0.0,0.0
16231,2017-08-06 07:00:00,8722.9,22414.2,2248.1,4799.7,4705.3,28.00,10.25,33.20,30.80,0.0,0.0
15804,2017-07-19 12:00:00,4952.3,37688.2,6862.0,4809.7,4705.3,52.15,38.52,51.46,50.56,1.0,0.0
8685,2016-09-25 21:00:00,4530.5,28131.9,4916.4,4293.6,3791.1,49.03,39.98,49.58,49.19,0.0,0.0
19183,2017-12-07 06:00:00,3834.6,25828.1,7235.0,4182.0,4147.8,61.19,61.19,61.19,69.09,0.0,1.0
16068,2017-07-30 12:00:00,2183.7,28384.2,5249.4,4809.7,4620.3,43.40,10.53,43.42,46.22,0.0,0.0
5204,2016-05-03 20:00:00,7302.6,29327.0,852.2,3456.3,4354.3,32.10,32.10,32.10,32.10,1.0,1.0
2854,2016-01-26 21:00:00,3050.7,35444.7,7986.6,4529.7,4705.3,53.12,27.12,49.12,50.62,1.0,0.0


#### 3) MW Carbón disponible

Sumamos la potencia disponible de carbón. Nos interesa el total de MW disponibles.

In [16]:
df_esios['POT_DISP_CARBON']=[x['POT_DISP_HULLA_ANT'] + x['POT_DISP_HULA_SB'] for _,x in df_esios.iterrows()]

In [17]:
df_esios.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80,0.0,0.0,10060.7
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13,0.0,0.0,10060.7
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96,0.0,0.0,10060.7
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01,0.0,0.0,10060.7
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42,0.0,0.0,10060.7


#### 4) Datos pasados

Necesitamos el carbón casado el día anterior (ya que el carbón que se va a casar ese día no lo conocemos) así que tendremos que restar 1 día a la fecha para la columna P48_CARBON, creando la columna adicional P48_CARBON_D-1

Aprovechamos esta operación para crear la columna RESULTADO D-1 que es la columna reserva del día anterior.

In [18]:
df_esios_ant= df_esios[['datetime','P48_CARBON','RESERVA']].copy()

In [19]:
df_esios_ant.head()

,datetime,P48_CARBON,RESERVA
0,2015-09-30 00:00:00,7181.4,0.0
1,2015-09-30 01:00:00,6771.7,0.0
2,2015-09-30 02:00:00,6559.5,0.0
3,2015-09-30 03:00:00,6560.4,0.0
4,2015-09-30 04:00:00,6724.1,0.0


In [20]:
df_esios_ant.dtypes

datetime      datetime64[ns]
P48_CARBON           float64
RESERVA              float64
dtype: object

In [21]:
dia=timedelta(days=1) 

fecha=pd.to_datetime(df_esios_ant['datetime'])
fecha_ant=pd.Series([x + dia for x in fecha])
df_esios_ant['datetime']=fecha_ant
df_esios_ant.tail()

,datetime,P48_CARBON,RESERVA
26323,2018-10-01 19:00:00,4861.1,0.0
26324,2018-10-01 20:00:00,4782.9,0.0
26325,2018-10-01 21:00:00,4826.0,0.0
26326,2018-10-01 22:00:00,4169.8,1.0
26327,2018-10-01 23:00:00,3775.5,1.0


In [22]:
df_esios_ant.head()

,datetime,P48_CARBON,RESERVA
0,2015-10-01 00:00:00,7181.4,0.0
1,2015-10-01 01:00:00,6771.7,0.0
2,2015-10-01 02:00:00,6559.5,0.0
3,2015-10-01 03:00:00,6560.4,0.0
4,2015-10-01 04:00:00,6724.1,0.0


In [23]:
df_esios_ant.columns=['datetime','P48_CARBON_D-1','RESERVA_D-1']

In [24]:
df_esios_ant.head()

,datetime,P48_CARBON_D-1,RESERVA_D-1
0,2015-10-01 00:00:00,7181.4,0.0
1,2015-10-01 01:00:00,6771.7,0.0
2,2015-10-01 02:00:00,6559.5,0.0
3,2015-10-01 03:00:00,6560.4,0.0
4,2015-10-01 04:00:00,6724.1,0.0


In [25]:
df_esios2=df_esios.merge(df_esios_ant,how='left',on='datetime')
df_esios2.head(48)

Esperamos a unir todos los datos para comenzar con el tratamiento de duplicados, NaN etc.

#### 5) Num centrales en D-1

Está guardado en el fichero .csv *carbon_dia_ant.csv* en la carpeta data.

Cuando se descargaron del csv original (P48) que contenía el número de centrales por día, se transformó a un nuevo fichero *carbón_dia_ant.csv* (en la misma carpeta) en el que se muestra el número de centrales arrancadas el día anterior. Para ello se programó el notebook carbón_D-1.ypynb

El periodo ya está adaptado a la fecha inicial y final de nuestro conjunto


In [39]:
df_centrales_carbon=pd.read_csv('../data/carbon_dia_ant.csv', sep=';')
df_centrales_carbon.head()

,datetime,NUM_PLANTAS_D-1
0,2015-10-01 00:00:00,22
1,2015-10-01 01:00:00,22
2,2015-10-01 02:00:00,22
3,2015-10-01 03:00:00,23
4,2015-10-01 04:00:00,23


In [40]:
df_centrales_carbon.tail()

,datetime,NUM_PLANTAS_D-1
26299,2018-09-30 19:00:00,17
26300,2018-09-30 20:00:00,17
26301,2018-09-30 21:00:00,17
26302,2018-09-30 22:00:00,17
26303,2018-09-30 23:00:00,17


In [41]:
df_centrales_carbon.dtypes

datetime           object
NUM_PLANTAS_D-1     int64
dtype: object

In [42]:
#Ponemos datetime en formato fecha
df_centrales_carbon['datetime']=pd.to_datetime(df_centrales_carbon['datetime'])
df_centrales_carbon.dtypes

datetime           datetime64[ns]
NUM_PLANTAS_D-1             int64
dtype: object

In [87]:
df_esios3=df_esios2.merge(df_centrales_carbon,how='left',on='datetime')

In [88]:
df_esios3.head(35)

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON,P48_CARBON_D-1,RESERVA_D-1,NUM_PLANTAS_D-1
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80,0.0,0.0,10060.7,NaN,NaN,NaN
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13,0.0,0.0,10060.7,NaN,NaN,NaN
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96,0.0,0.0,10060.7,NaN,NaN,NaN
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01,0.0,0.0,10060.7,NaN,NaN,NaN
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42,0.0,0.0,10060.7,NaN,NaN,NaN
5,2015-09-30 05:00:00,2777.9,22370.5,7174.6,5355.4,4705.3,45.65,30.55,50.65,50.65,0.0,0.0,10060.7,NaN,NaN,NaN
6,2015-09-30 06:00:00,2522.1,24532.6,7955.3,5355.4,4705.3,52.00,42.93,56.13,54.31,0.0,0.0,10060.7,NaN,NaN,NaN
7,2015-09-30 07:00:00,2303.5,27986.9,8281.3,5355.4,4705.3,59.90,53.78,62.00,62.40,0.0,0.0,10060.7,NaN,NaN,NaN
8,2015-09-30 08:00:00,2141.9,29690.2,8304.3,5355.4,4705.3,60.01,55.50,61.71,62.51,0.0,0.0,10060.7,NaN,NaN,NaN
9,2015-09-30 09:00:00,1980.8,30192.1,8263.3,5355.4,4705.3,60.60,49.27,62.86,63.03,0.0,0.0,10060.7,NaN,NaN,NaN


In [89]:
df_esios3.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON,P48_CARBON_D-1,RESERVA_D-1,NUM_PLANTAS_D-1
count,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26310.000000,26310.000000,26310.000000
mean,5476.502122,28669.419820,4412.931435,4421.154042,4188.930546,48.742054,42.101521,48.469277,48.480474,0.575350,0.247447,8610.084588,4413.362269,0.575181,13.706803
std,3185.554082,4617.498898,2217.550000,474.642811,469.394799,14.950116,20.789213,15.092187,15.105922,0.494299,0.431537,752.717789,2218.445416,0.494325,6.262792
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.000000,0.000000,0.000000,0.000000,5693.700000,546.000000,0.000000,2.000000
25%,3015.300000,24730.200000,2318.000000,4125.800000,3797.900000,40.140000,29.750000,40.050000,40.100000,0.000000,0.000000,8120.400000,2317.575000,0.000000,8.000000
50%,4820.500000,28919.100000,4636.400000,4483.200000,4148.100000,49.550000,39.160000,49.000000,48.960000,1.000000,0.000000,8679.100000,4637.250000,1.000000,15.000000
75%,7286.100000,32265.500000,6286.100000,4799.700000,4705.300000,59.450000,51.550000,58.900000,58.740000,1.000000,0.000000,9153.600000,6288.950000,1.000000,19.000000
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.490000,103.610000,1.000000,1.000000,17752.000000,8768.300000,1.000000,25.000000


In [90]:
df_esios3.shape

(26337, 16)

#### 6) Hidraulicidad

Agregamos columna de reservas (Anuales e Hiperanuales) disponible en el fichero *embalses_dia.csv*, que es un tratamiento de los datos descargados directamente de REE (*EMBALSES.csv*): https://www.ree.es/es/estadisticas-del-sistema-electrico-espanol/series-estadisticas/series-estadisticas-nacionales gracias al notebook embalses.ipynb.

Los datos son diarios. Para convertirlos a horarios asumimos el mismo valor en todos los periodos de un día, igual que se hizo para pasar de mensual a diario.

In [48]:
df_embalses=pd.read_csv('../data/embalses_dia.csv', sep=';')
df_embalses.head()

,datetime,mes_ano,MES,Reservas embalses anuales (GWh),Reservas embalses hiperanuales (GWh)
0,2015-10-01,10-15,oct-15,3.714,4.909
1,2015-10-02,10-15,oct-15,3.714,4.909
2,2015-10-03,10-15,oct-15,3.714,4.909
3,2015-10-04,10-15,oct-15,3.714,4.909
4,2015-10-05,10-15,oct-15,3.714,4.909


In [91]:
df_data=df_esios3.copy()
df_data.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON,P48_CARBON_D-1,RESERVA_D-1,NUM_PLANTAS_D-1
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80,0.0,0.0,10060.7,NaN,NaN,NaN
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13,0.0,0.0,10060.7,NaN,NaN,NaN
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96,0.0,0.0,10060.7,NaN,NaN,NaN
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01,0.0,0.0,10060.7,NaN,NaN,NaN
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42,0.0,0.0,10060.7,NaN,NaN,NaN


In [92]:
#Creamos columna mes_ano en dataframe df_esios3 para hacer el merge
df_data['mes_ano']=[str(x.month) + '-' + str(x.year)[-2:] for x in df_data['datetime']]
df_data.sample(10)

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON,P48_CARBON_D-1,RESERVA_D-1,NUM_PLANTAS_D-1,mes_ano
13870,2017-04-29 16:00:00,7416.6,25864.5,2424.9,3835.8,3445.1,38.67,27.46,38.67,43.67,0.0,0.0,7280.9,4144.9,1.0,11.0,4-17
14857,2017-06-09 19:00:00,2563.2,31632.6,5678.0,3984.1,4135.7,52.48,35.07,54.45,54.70,0.0,0.0,8119.8,6067.6,1.0,17.0,6-17
15138,2017-06-21 12:00:00,741.0,38032.0,7553.5,4038.9,4608.1,58.99,51.60,59.49,59.00,0.0,0.0,8647.0,8212.5,0.0,23.0,6-17
11690,2017-01-28 19:00:00,6727.5,32715.6,6488.6,3836.7,4147.8,80.00,80.00,80.48,80.00,0.0,1.0,7984.5,8066.6,0.0,24.0,1-17
23604,2018-06-09 03:00:00,2431.7,22229.7,2977.4,4182.0,3783.9,61.10,32.13,55.57,51.38,1.0,0.0,7965.9,4169.4,1.0,12.0,6-18
15596,2017-07-10 14:00:00,3076.9,35217.4,6840.6,4809.7,4135.3,54.30,44.18,53.30,52.78,1.0,0.0,8945.0,5499.9,1.0,18.0,7-17
1708,2015-12-10 00:00:00,3603.3,26660.5,7974.8,5004.8,4705.3,63.29,41.85,63.29,60.90,1.0,0.0,9710.1,7368.5,1.0,21.0,12-15
19470,2017-12-18 20:00:00,10110.5,38719.7,8159.4,4263.9,4705.3,73.10,73.10,69.32,68.61,1.0,1.0,8969.2,7601.9,1.0,22.0,12-17
17122,2017-09-12 04:00:00,4689.4,23247.6,2997.2,4829.4,4705.3,42.82,11.48,39.00,38.00,1.0,0.0,9534.7,1558.0,1.0,8.0,9-17
3012,2016-02-02 08:00:00,1240.5,33681.2,7160.9,4687.9,4705.3,46.69,31.60,44.26,45.89,1.0,0.0,9393.2,6424.8,1.0,21.0,2-16


In [93]:
df_data.dtypes

datetime              datetime64[ns]
P48_EOLICA                   float64
P48_DEMANDA                  float64
P48_CARBON                   float64
POT_DISP_HULLA_ANT           float64
POT_DISP_HULA_SB             float64
PRECIO_MD_ESP                float64
PRECIO_MD_FR                 float64
PRECIO_I1                    float64
PRECIO_I2                    float64
RESERVA                      float64
ACOPLADO_FR                  float64
POT_DISP_CARBON              float64
P48_CARBON_D-1               float64
RESERVA_D-1                  float64
NUM_PLANTAS_D-1              float64
mes_ano                       object
dtype: object

In [94]:
df_data.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON,P48_CARBON_D-1,RESERVA_D-1,NUM_PLANTAS_D-1
count,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26337.000000,26310.000000,26310.000000,26310.000000
mean,5476.502122,28669.419820,4412.931435,4421.154042,4188.930546,48.742054,42.101521,48.469277,48.480474,0.575350,0.247447,8610.084588,4413.362269,0.575181,13.706803
std,3185.554082,4617.498898,2217.550000,474.642811,469.394799,14.950116,20.789213,15.092187,15.105922,0.494299,0.431537,752.717789,2218.445416,0.494325,6.262792
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.000000,0.000000,0.000000,0.000000,5693.700000,546.000000,0.000000,2.000000
25%,3015.300000,24730.200000,2318.000000,4125.800000,3797.900000,40.140000,29.750000,40.050000,40.100000,0.000000,0.000000,8120.400000,2317.575000,0.000000,8.000000
50%,4820.500000,28919.100000,4636.400000,4483.200000,4148.100000,49.550000,39.160000,49.000000,48.960000,1.000000,0.000000,8679.100000,4637.250000,1.000000,15.000000
75%,7286.100000,32265.500000,6286.100000,4799.700000,4705.300000,59.450000,51.550000,58.900000,58.740000,1.000000,0.000000,9153.600000,6288.950000,1.000000,19.000000
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.490000,103.610000,1.000000,1.000000,17752.000000,8768.300000,1.000000,25.000000


In [97]:
df_data['HID_ANUAL']=np.zeros(len(df_esios3['datetime']))
df_data['HID_HIPERAN']=np.zeros(len(df_esios3['datetime']))


In [98]:
df_data.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON,P48_CARBON_D-1,RESERVA_D-1,NUM_PLANTAS_D-1,mes_ano,HID_ANUAL,HID_HIPERAN
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80,0.0,0.0,10060.7,NaN,NaN,NaN,9-15,0.0,0.0
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13,0.0,0.0,10060.7,NaN,NaN,NaN,9-15,0.0,0.0
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96,0.0,0.0,10060.7,NaN,NaN,NaN,9-15,0.0,0.0
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01,0.0,0.0,10060.7,NaN,NaN,NaN,9-15,0.0,0.0
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42,0.0,0.0,10060.7,NaN,NaN,NaN,9-15,0.0,0.0


In [114]:
#comprobamos NaN por días 25-23 horas descuadrados

df_esios2[pd.isna(df_esios2['P48_CARBON_D-1'])]

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON,P48_CARBON_D-1,RESERVA_D-1
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80,0.0,0.0,10060.7,NaN,NaN
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13,0.0,0.0,10060.7,NaN,NaN
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96,0.0,0.0,10060.7,NaN,NaN
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01,0.0,0.0,10060.7,NaN,NaN
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42,0.0,0.0,10060.7,NaN,NaN
5,2015-09-30 05:00:00,2777.9,22370.5,7174.6,5355.4,4705.3,45.65,30.55,50.65,50.65,0.0,0.0,10060.7,NaN,NaN
6,2015-09-30 06:00:00,2522.1,24532.6,7955.3,5355.4,4705.3,52.00,42.93,56.13,54.31,0.0,0.0,10060.7,NaN,NaN
7,2015-09-30 07:00:00,2303.5,27986.9,8281.3,5355.4,4705.3,59.90,53.78,62.00,62.40,0.0,0.0,10060.7,NaN,NaN
8,2015-09-30 08:00:00,2141.9,29690.2,8304.3,5355.4,4705.3,60.01,55.50,61.71,62.51,0.0,0.0,10060.7,NaN,NaN
9,2015-09-30 09:00:00,1980.8,30192.1,8263.3,5355.4,4705.3,60.60,49.27,62.86,63.03,0.0,0.0,10060.7,NaN,NaN


In [30]:
df_esios2[df_esios2['datetime'].duplicated()]['datetime']

603     2015-10-25 02:00:00
628     2015-10-26 02:00:00
9508    2016-10-30 02:00:00
9533    2016-10-31 02:00:00
18245   2017-10-29 02:00:00
18270   2017-10-30 02:00:00
Name: datetime, dtype: datetime64[ns]